In [ ]:
import pandas as pd
import matplotlib
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import openpyxl
from openpyxl import Workbook
import csv 
import os
import shutil
import xlsxwriter 
# import datashader as ds
# import datashader.transfer_functions as tf

In [ ]:
import plotly.io as pio
pio.renderers.default = 'notebook'
import plotly.express as px

In [ ]:
file_path_categorical = os.path.expanduser("~/Documents/Python Scripts/widsdatathon2025/TRAIN/TRAIN_CATEGORICAL_METADATA.xlsx")
file_path_matrices = os.path.expanduser("~/Documents/Python Scripts/widsdatathon2025/TRAIN/TRAIN_FUNCTIONAL_CONNECTOME_MATRICES.csv")
file_path_quantitative = os.path.expanduser("~/Documents/Python Scripts/widsdatathon2025/TRAIN/TRAIN_QUANTITATIVE_METADATA.xlsx")
file_path_solutions = os.path.expanduser("~/Documents/Python Scripts/widsdatathon2025/TRAIN/TRAINING_SOLUTIONS.xlsx")

# Read the Excel file
# pd.read_csv(pizza_types)
df_categorical = pd.read_excel(file_path_categorical)
df_matrices = pd.read_csv(file_path_matrices, header=[0], index_col=[0])
df_quant = pd.read_excel(file_path_quantitative)
df_solutions = pd.read_excel(file_path_solutions)

# Print the DataFrame

print(df_matrices)

In [ ]:
df_matrices.describe()

In [ ]:
newdf = df_matrices.merge(df_solutions, how='inner', on = 'participant_id')
print(newdf)

In [ ]:
#first attempt at heatmap (it breaks kaggle down so try it on your own)
fig = px.imshow(df_matrices, aspect='auto', color_continuous_scale='viridis')
fig.show()

In [ ]:
#aggregated the data by every 10 columns to reduce the data size for the heatmap
aggregated_df = df_matrices.groupby(np.arange(df_matrices.shape[1]) // 10, axis=1).mean(numeric_only =True)


In [ ]:
fig = px.imshow(aggregated_df, aspect='auto', color_continuous_scale='viridis')
fig.show()

In [ ]:
#trying to understand which values are correlated with each other in the matrix. Didn't really understand the results?
np.fill_diagonal(aggregated_df.values, np.nan)

# Unstack and sort by absolute correlation value
corr_pairs = aggregated_df.unstack().sort_values(ascending=False, key=abs)

# Display the top correlation pairs
top_n = 1000
top_corr_pairs = corr_pairs.head(top_n)
for (var1, var2), corr_value in top_corr_pairs.items():
    print(f"Variables: {var1} and {var2} have a correlation of {corr_value:.2f}")

In [ ]:
#filtering the dataset to show only rows that either have ADHD or Women 
filtered_ADHD_Female = newdf.loc[(newdf['ADHD_Outcome'] == 1)| (newdf['Sex_F'] == 1)]
# filtered.info()
#& (newdf['Sex_F'] == '1')
print(filtered_ADHD_Female)

In [ ]:
filtered_ADHD_Female.set_index('participant_id', inplace=True)
#setting the index

In [ ]:
#heatmap of the filtered dataset. Kaggle will die if you run this...
fig = px.imshow(filtered_ADHD_Female, aspect='auto', color_continuous_scale='viridis')
fig.show()

In [ ]:
#aggregated datset of the filtered dataframe
aggregated_df_filtered = filtered_ADHD_Female.groupby(np.arange(filtered_ADHD_Female.shape[1]) // 10, axis=1).mean(numeric_only =True)
fig = px.imshow(aggregated_df_filtered, aspect='auto', color_continuous_scale='viridis')
fig.show()

In [ ]:
df_matrices.set_index('participant_id', inplace=True)

In [ ]:
#Getting the top correlated values for each column
# Function to get top N column names
def top_n_columns(row, n=3):
    return row.nlargest(n).index.tolist()

# Apply function to each row
top_columns = df_matrices.apply(top_n_columns, axis=1, n=3)
top_columns_df = pd.DataFrame(top_columns.tolist(), columns=[f'Top {i+1}' for i in range(3)])

# Combine with original DataFrame
# Reset indices to ensure alignment
df_reset = df_matrices.reset_index(drop=True)
top_columns_df_reset = top_columns_df.reset_index(drop=True)

# Combine with original DataFrame
result_df = pd.concat([df_reset, top_columns_df_reset], axis=1)
print(result_df)

In [ ]:
# appending the top values to the df Matrices data merged with the solutions data
newdf['Top 1'] = result_df['Top 1']
newdf['Top 2'] = result_df['Top 2']
newdf['Top 3'] = result_df['Top 3']

newdf

In [ ]:
# Columns to export
columns_to_export = ['Top 1', 'Top 2', 'Top 3', 'Sex_F', 'ADHD_Outcome']

# Export to Excel, including only the selected columns to analyze further 
newdf.to_excel('output2.xlsx', columns=columns_to_export, index=False)
